In [6]:
import pygame
import random
import math
import sys
import os

# ================== CONFIG ==================
WIDTH, HEIGHT = 1000, 700
FPS = 60

# Colors
WHITE = (240, 240, 240)
RED = (255, 70, 70)
GREEN = (70, 255, 120)
BLUE = (80, 180, 255)
CYAN = (0, 255, 220)
YELLOW = (255, 220, 100)
PURPLE = (180, 120, 255)
BG = (10, 10, 30)

# ================== INIT ==================
pygame.init()
pygame.mixer.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("🚀 Space Shooter – Portfolio Project")
clock = pygame.time.Clock()
font = pygame.font.SysFont("consolas", 20)
big_font = pygame.font.SysFont("consolas", 50)

# ================== SAFE SOUND LOADER ==================
def load_sound(path):
    if os.path.exists(path):
        return pygame.mixer.Sound(path)
    return None

shoot_snd = load_sound("shoot.wav")
explode_snd = load_sound("explode.wav")
power_snd = load_sound("power.wav")

# ================== GAME STATES ==================
MENU = "menu"
PLAYING = "playing"
PAUSED = "paused"
GAME_OVER = "game_over"
state = MENU

# ================== UTIL ==================
def clamp(v, a, b):
    return max(a, min(v, b))

def draw_text(txt, x, y, c=WHITE, f=font):
    screen.blit(f.render(txt, True, c), (x, y))

# ================== STARFIELD ==================
class Star:
    def __init__(self):
        self.x = random.randint(0, WIDTH)
        self.y = random.randint(0, HEIGHT)
        self.speed = random.uniform(0.5, 2)
        self.size = random.randint(1, 3)

    def update(self):
        self.y += self.speed
        if self.y > HEIGHT:
            self.y = 0
            self.x = random.randint(0, WIDTH)

    def draw(self):
        pygame.draw.circle(screen, (200, 200, 255), (int(self.x), int(self.y)), self.size)

stars = [Star() for _ in range(150)]

# ================== EFFECTS ==================
particles = []
shake = 0
flash = 0

class Particle:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.vx = random.uniform(-3, 3)
        self.vy = random.uniform(-3, 3)
        self.life = random.randint(20, 40)

    def update(self):
        self.x += self.vx
        self.y += self.vy
        self.life -= 1

    def draw(self):
        if self.life > 0:
            pygame.draw.circle(screen, YELLOW, (int(self.x), int(self.y)), 2)

# ================== DRAW HELPERS ==================
def draw_glow(pos, color, radius):
    glow = pygame.Surface((radius*2, radius*2), pygame.SRCALPHA)
    pygame.draw.circle(glow, (*color, 80), (radius, radius), radius)
    screen.blit(glow, (pos[0]-radius, pos[1]-radius))

def draw_bar(x, y, val, maxv, color):
    pygame.draw.rect(screen, (40,40,40), (x, y, 200, 18))
    pygame.draw.rect(screen, color, (x, y, 200*(val/maxv), 18))

def draw_pixel_ship(x, y, scale=5, color=CYAN):
    shape = [
        "00100",
        "01110",
        "11111",
        "01110",
        "00100"
    ]
    for r, row in enumerate(shape):
        for c, ch in enumerate(row):
            if ch == "1":
                pygame.draw.rect(
                    screen, color,
                    (x + c*scale, y + r*scale, scale, scale)
                )

def draw_enemy_ship(x, y, scale=5):
    shape = [
        "01110",
        "11111",
        "10101",
        "00100"
    ]
    for r, row in enumerate(shape):
        for c, ch in enumerate(row):
            if ch == "1":
                pygame.draw.rect(
                    screen, RED,
                    (x + c*scale, y + r*scale, scale, scale)
                )

# ================== ENTITIES ==================
class Bullet:
    def __init__(self, x, y, dy, color):
        self.rect = pygame.Rect(x, y, 6, 14)
        self.dy = dy
        self.color = color

    def update(self):
        self.rect.y += self.dy

    def draw(self):
        draw_glow(self.rect.center, self.color, 8)
        pygame.draw.rect(screen, self.color, self.rect)

class Player:
    def __init__(self):
        self.rect = pygame.Rect(WIDTH//2, HEIGHT-80, 25, 25)
        self.health = 100
        self.bullets = []
        self.cooldown = 0
        self.score = 0
        self.rapid = 0
        self.shield = 0

    def move(self, keys):
        if keys[pygame.K_a]: self.rect.x -= 5
        if keys[pygame.K_d]: self.rect.x += 5
        if keys[pygame.K_w]: self.rect.y -= 5
        if keys[pygame.K_s]: self.rect.y += 5
        self.rect.x = clamp(self.rect.x, 0, WIDTH)
        self.rect.y = clamp(self.rect.y, 0, HEIGHT)

    def shoot(self):
        if self.cooldown == 0:
            self.bullets.append(Bullet(self.rect.x+10, self.rect.y, -10, CYAN))
            if shoot_snd: shoot_snd.play()
            self.cooldown = 5 if self.rapid else 15

    def update(self):
        if self.cooldown > 0: self.cooldown -= 1
        if self.rapid > 0: self.rapid -= 1
        if self.shield > 0: self.shield -= 1
        for b in self.bullets[:]:
            b.update()
            if b.rect.bottom < 0:
                self.bullets.remove(b)

    def draw(self):
        draw_pixel_ship(self.rect.x, self.rect.y)
        if self.shield:
            draw_glow(self.rect.center, BLUE, 20)
        for b in self.bullets:
            b.draw()

class Enemy:
    def __init__(self, speed):
        self.rect = pygame.Rect(random.randint(0, WIDTH-30), -40, 25, 25)
        self.hp = 30
        self.speed = speed

    def update(self, player):
        dx = player.rect.centerx - self.rect.centerx
        dy = player.rect.centery - self.rect.centery
        dist = math.hypot(dx, dy) or 1
        self.rect.x += int(dx/dist * self.speed)
        self.rect.y += int(dy/dist * self.speed)

    def draw(self):
        draw_enemy_ship(self.rect.x, self.rect.y)

# ================== GAME DATA ==================
player = Player()
enemies = []
level = 1
spawn_timer = 0

# ================== MAIN LOOP ==================
running = True
while running:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if state == MENU and event.key == pygame.K_RETURN:
                player = Player()
                enemies.clear()
                level = 1
                state = PLAYING
            elif state == GAME_OVER and event.key == pygame.K_RETURN:
                state = MENU
            elif event.key == pygame.K_p and state == PLAYING:
                state = PAUSED
            elif event.key == pygame.K_p and state == PAUSED:
                state = PLAYING

    screen.fill(BG)

    for s in stars:
        s.update()
        s.draw()

    if state == MENU:
        draw_text("SPACE SHOOTER", WIDTH//2-150, HEIGHT//2-100, CYAN, big_font)
        draw_text("Press ENTER to Start", WIDTH//2-120, HEIGHT//2)
    elif state == PAUSED:
        draw_text("PAUSED", WIDTH//2-60, HEIGHT//2, YELLOW, big_font)
    elif state == PLAYING:
        keys = pygame.key.get_pressed()
        player.move(keys)
        if keys[pygame.K_SPACE]:
            player.shoot()

        spawn_timer += 1
        if spawn_timer > max(20, 60 - level*5):
            enemies.append(Enemy(1.5 + level*0.3))
            spawn_timer = 0

        player.update()

        for e in enemies[:]:
            e.update(player)
            for b in player.bullets[:]:
                if e.rect.colliderect(b.rect):
                    e.hp -= 15
                    player.bullets.remove(b)
                    particles += [Particle(*e.rect.center) for _ in range(8)]
                    if e.hp <= 0:
                        enemies.remove(e)
                        player.score += 1
                        if explode_snd: explode_snd.play()
            if e.rect.colliderect(player.rect):
                if not player.shield:
                    player.health -= 10
                enemies.remove(e)

        if player.score > level * 10:
            level += 1

        player.draw()
        for e in enemies:
            e.draw()

        draw_bar(20, 20, player.health, 100, GREEN)
        draw_text(f"Score: {player.score}", 20, 50)
        draw_text(f"Level: {level}", WIDTH-120, 20)

        if player.health <= 0:
            state = GAME_OVER

    elif state == GAME_OVER:
        draw_text("GAME OVER", WIDTH//2-120, HEIGHT//2-50, RED, big_font)
        draw_text("Press ENTER for Menu", WIDTH//2-150, HEIGHT//2+10)

    for p in particles[:]:
        p.update()
        p.draw()
        if p.life <= 0:
            particles.remove(p)

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [7]:
import pygame
import random
import sys
import math

# ================= CONFIG =================
WIDTH, HEIGHT = 900, 700
FPS = 60

# Colors
WHITE = (230,230,230)
CYAN = (0,255,220)
RED = (255,70,70)
YELLOW = (255,220,100)
GREEN = (80,255,120)
BLUE = (100,180,255)
BG = (5,5,20)

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Shooter – Professional 2D")
clock = pygame.time.Clock()
font = pygame.font.SysFont("consolas", 18)
big_font = pygame.font.SysFont("consolas", 48)

# ================= GAME STATES =================
MENU = "menu"
PLAYING = "playing"
GAME_OVER = "game_over"
state = MENU

# ================= STARFIELD =================
class Star:
    def __init__(self, speed):
        self.x = random.randint(0, WIDTH)
        self.y = random.randint(0, HEIGHT)
        self.speed = speed
        self.size = random.randint(1, 3)

    def update(self):
        self.y += self.speed
        if self.y > HEIGHT:
            self.y = 0
            self.x = random.randint(0, WIDTH)

    def draw(self):
        pygame.draw.circle(screen, (180,180,255), (int(self.x), int(self.y)), self.size)

stars = [Star(random.uniform(0.5,2.5)) for _ in range(160)]

# ================= PARTICLES =================
particles = []

class Particle:
    def __init__(self, x, y):
        self.x, self.y = x, y
        self.vx = random.uniform(-3,3)
        self.vy = random.uniform(-3,3)
        self.life = random.randint(20,40)

    def update(self):
        self.x += self.vx
        self.y += self.vy
        self.life -= 1

    def draw(self):
        if self.life > 0:
            pygame.draw.circle(screen, YELLOW, (int(self.x), int(self.y)), 2)

# ================= DRAW HELPERS =================
def draw_glow(pos, color, radius):
    surf = pygame.Surface((radius*2, radius*2), pygame.SRCALPHA)
    pygame.draw.circle(surf, (*color, 90), (radius, radius), radius)
    screen.blit(surf, (pos[0]-radius, pos[1]-radius))

def draw_player(x, y):
    pygame.draw.polygon(
        screen, CYAN,
        [(x, y-18), (x-16, y+16), (x+16, y+16)]
    )
    draw_glow((x,y), CYAN, 18)

def draw_enemy(x, y):
    pygame.draw.polygon(
        screen, RED,
        [(x-14, y-10), (x+14, y-10), (x, y+18)]
    )

# ================= ENTITIES =================
class Bullet:
    def __init__(self, x, y, speed, color):
        self.rect = pygame.Rect(x, y, 4, 12)
        self.speed = speed
        self.color = color

    def update(self):
        self.rect.y += self.speed

    def draw(self):
        draw_glow(self.rect.center, self.color, 10)
        pygame.draw.rect(screen, self.color, self.rect)

class Player:
    def __init__(self):
        self.x = WIDTH//2
        self.y = HEIGHT-80
        self.health = 100
        self.bullets = []
        self.cooldown = 0
        self.score = 0

    def move(self, keys):
        if keys[pygame.K_a] or keys[pygame.K_LEFT]: self.x -= 5
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]: self.x += 5
        if keys[pygame.K_w] or keys[pygame.K_UP]: self.y -= 5
        if keys[pygame.K_s] or keys[pygame.K_DOWN]: self.y += 5
        self.x = max(20, min(WIDTH-20, self.x))
        self.y = max(20, min(HEIGHT-20, self.y))

    def shoot(self):
        if self.cooldown == 0:
            self.bullets.append(Bullet(self.x-2, self.y-20, -10, CYAN))
            self.cooldown = 12

    def update(self):
        if self.cooldown > 0:
            self.cooldown -= 1
        for b in self.bullets[:]:
            b.update()
            if b.rect.bottom < 0:
                self.bullets.remove(b)

    def draw(self):
        draw_player(self.x, self.y)
        for b in self.bullets:
            b.draw()

class Enemy:
    def __init__(self, speed):
        self.x = random.randint(40, WIDTH-40)
        self.y = -30
        self.speed = speed
        self.hp = 30

    def update(self):
        self.y += self.speed

    def draw(self):
        draw_enemy(self.x, self.y)

# ================= GAME DATA =================
player = Player()
enemies = []
spawn_timer = 0
level = 1

# ================= MAIN LOOP =================
running = True
while running:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if state == MENU and event.key == pygame.K_RETURN:
                player = Player()
                enemies.clear()
                particles.clear()
                level = 1
                state = PLAYING
            elif state == GAME_OVER and event.key == pygame.K_RETURN:
                state = MENU

    screen.fill(BG)

    # Background
    for s in stars:
        s.update()
        s.draw()

    if state == MENU:
        screen.blit(big_font.render("SPACE SHOOTER", True, CYAN), (WIDTH//2-200, HEIGHT//2-120))
        screen.blit(font.render("Press ENTER to Start", True, WHITE), (WIDTH//2-110, HEIGHT//2))

    elif state == PLAYING:
        keys = pygame.key.get_pressed()
        player.move(keys)
        if keys[pygame.K_SPACE]:
            player.shoot()

        spawn_timer += 1
        if spawn_timer > max(20, 60 - level*5):
            enemies.append(Enemy(2 + level*0.3))
            spawn_timer = 0

        player.update()

        for e in enemies[:]:
            e.update()
            for b in player.bullets[:]:
                if abs(e.x - b.rect.centerx) < 20 and abs(e.y - b.rect.centery) < 20:
                    e.hp -= 15
                    player.bullets.remove(b)
                    particles += [Particle(e.x, e.y) for _ in range(8)]
                    if e.hp <= 0:
                        enemies.remove(e)
                        player.score += 1
            if abs(e.x - player.x) < 20 and abs(e.y - player.y) < 20:
                player.health -= 15
                enemies.remove(e)

        if player.score > level * 10:
            level += 1

        player.draw()
        for e in enemies:
            e.draw()

        # UI
        pygame.draw.rect(screen, (40,40,40), (20,20,200,18))
        pygame.draw.rect(screen, GREEN, (20,20,200*(player.health/100),18))
        screen.blit(font.render(f"Score: {player.score}", True, WHITE), (20,45))
        screen.blit(font.render(f"Level: {level}", True, WHITE), (WIDTH-120,20))

        if player.health <= 0:
            state = GAME_OVER

    elif state == GAME_OVER:
        screen.blit(big_font.render("GAME OVER", True, RED), (WIDTH//2-160, HEIGHT//2-80))
        screen.blit(font.render("Press ENTER to return to menu", True, WHITE), (WIDTH//2-170, HEIGHT//2))

    for p in particles[:]:
        p.update()
        p.draw()
        if p.life <= 0:
            particles.remove(p)

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 

In [3]:
import pygame
import random
import sys
import math

# ================== CONFIG ==================
WIDTH, HEIGHT = 960, 720
FPS = 60

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pixel Space Shooter – Professional 2D")
clock = pygame.time.Clock()

FONT = pygame.font.SysFont("consolas", 18)
BIG = pygame.font.SysFont("consolas", 42)

# Colors
WHITE = (230,230,230)
CYAN = (0,255,220)
RED = (255,80,80)
YELLOW = (255,220,100)
GREEN = (80,255,120)
BLUE = (120,180,255)
BG = (6,6,18)

# ================== GAME STATES ==================
MENU = "menu"
SELECT = "select"
PLAY = "play"
GAME_OVER = "game_over"
state = MENU

# ================== STARFIELD ==================
class Star:
    def __init__(self):
        self.x = random.randint(0, WIDTH)
        self.y = random.randint(0, HEIGHT)
        self.speed = random.uniform(0.5, 2.5)
        self.size = random.randint(1, 3)

    def update(self):
        self.y += self.speed
        if self.y > HEIGHT:
            self.y = 0
            self.x = random.randint(0, WIDTH)

    def draw(self):
        pygame.draw.circle(screen, (180,180,255),
                           (int(self.x), int(self.y)), self.size)

stars = [Star() for _ in range(200)]

# ================== PIXEL SHIPS ==================
def draw_ship(x, y, color):
    shape = [
        "0011100",
        "0111110",
        "1111111",
        "0111110",
        "0010100"
    ]
    scale = 4
    for r,row in enumerate(shape):
        for c,ch in enumerate(row):
            if ch == "1":
                pygame.draw.rect(
                    screen, color,
                    (x + c*scale, y + r*scale, scale, scale)
                )

def draw_enemy(x, y):
    shape = [
        "01110",
        "11111",
        "10101",
        "00100"
    ]
    scale = 4
    for r,row in enumerate(shape):
        for c,ch in enumerate(row):
            if ch == "1":
                pygame.draw.rect(
                    screen, RED,
                    (x + c*scale, y + r*scale, scale, scale)
                )

# ================== BULLETS ==================
class Bullet:
    def __init__(self, x, y, vx, vy, color, dmg):
        self.x, self.y = x, y
        self.vx, self.vy = vx, vy
        self.color = color
        self.dmg = dmg

    def update(self):
        self.x += self.vx
        self.y += self.vy

    def draw(self):
        pygame.draw.circle(screen, self.color,
                           (int(self.x), int(self.y)), 4)

# ================== PLAYER ==================
class Player:
    def __init__(self, weapon):
        self.x = WIDTH // 2
        self.y = HEIGHT - 90
        self.weapon = weapon
        self.hp = 100
        self.cooldown = 0
        self.bullets = []
        self.score = 0

    def shoot(self):
        if self.cooldown > 0:
            return
        if self.weapon == "laser":
            self.bullets.append(Bullet(self.x, self.y, 0, -10, CYAN, 10))
        elif self.weapon == "spread":
            for a in [-0.3, 0, 0.3]:
                self.bullets.append(
                    Bullet(self.x, self.y,
                           math.sin(a)*6, -10, YELLOW, 8)
                )
        elif self.weapon == "plasma":
            self.bullets.append(Bullet(self.x, self.y, 0, -6, BLUE, 20))
        self.cooldown = 15

    def update(self, keys):
        # WASD + ARROW KEYS
        if keys[pygame.K_a] or keys[pygame.K_LEFT]:
            self.x -= 5
        if keys[pygame.K_d] or keys[pygame.K_RIGHT]:
            self.x += 5
        if keys[pygame.K_w] or keys[pygame.K_UP]:
            self.y -= 5
        if keys[pygame.K_s] or keys[pygame.K_DOWN]:
            self.y += 5

        self.x = max(40, min(WIDTH-40, self.x))
        self.y = max(40, min(HEIGHT-40, self.y))

        if self.cooldown > 0:
            self.cooldown -= 1

        for b in self.bullets[:]:
            b.update()
            if b.y < 0:
                self.bullets.remove(b)

    def draw(self):
        draw_ship(self.x-14, self.y-14, CYAN)
        for b in self.bullets:
            b.draw()

# ================== ENEMY ==================
class Enemy:
    def __init__(self, speed):
        self.x = random.randint(40, WIDTH-40)
        self.y = -40
        self.hp = 30
        self.speed = speed

    def update(self):
        self.y += self.speed

    def draw(self):
        draw_enemy(self.x-10, self.y-10)

# ================== GAME DATA ==================
weapon_choice = "laser"
player = None
enemies = []
spawn_timer = 0
level = 1

# ================== MAIN LOOP ==================
running = True
while running:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if state == MENU and event.key == pygame.K_RETURN:
                state = SELECT

            elif state == SELECT:
                if event.key == pygame.K_1:
                    weapon_choice = "laser"
                if event.key == pygame.K_2:
                    weapon_choice = "spread"
                if event.key == pygame.K_3:
                    weapon_choice = "plasma"
                if event.key == pygame.K_RETURN:
                    player = Player(weapon_choice)
                    enemies.clear()
                    level = 1
                    state = PLAY

            elif state == GAME_OVER and event.key == pygame.K_RETURN:
                state = MENU

    screen.fill(BG)

    for s in stars:
        s.update()
        s.draw()

    # ================== MENU ==================
    if state == MENU:
        screen.blit(BIG.render("PIXEL SPACE SHOOTER", True, CYAN), (200, 240))
        screen.blit(FONT.render("MOVE: WASD or Arrow Keys", True, WHITE), (350, 310))
        screen.blit(FONT.render("SHOOT: SPACE", True, WHITE), (350, 335))
        screen.blit(FONT.render("WEAPON: 1 / 2 / 3", True, WHITE), (350, 360))
        screen.blit(FONT.render("Press ENTER to Start", True, GREEN), (360, 400))

    # ================== SELECT ==================
    elif state == SELECT:
        screen.blit(BIG.render("SELECT WEAPON", True, CYAN), (300, 200))
        screen.blit(FONT.render("1 - Laser (Fast)", True, WHITE), (380, 280))
        screen.blit(FONT.render("2 - Spread (Wide)", True, WHITE), (380, 310))
        screen.blit(FONT.render("3 - Plasma (Heavy)", True, WHITE), (380, 340))
        screen.blit(FONT.render("Press ENTER to Begin", True, GREEN), (350, 390))

    # ================== PLAY ==================
    elif state == PLAY:
        keys = pygame.key.get_pressed()
        if keys[pygame.K_SPACE]:
            player.shoot()

        player.update(keys)

        spawn_timer += 1
        if spawn_timer > max(30, 80 - level*5):
            enemies.append(Enemy(2 + level*0.3))
            spawn_timer = 0

        for e in enemies[:]:
            e.update()
            for b in player.bullets[:]:
                if abs(e.x - b.x) < 20 and abs(e.y - b.y) < 20:
                    e.hp -= b.dmg
                    player.bullets.remove(b)
                    if e.hp <= 0:
                        enemies.remove(e)
                        player.score += 1
            if abs(e.x - player.x) < 20 and abs(e.y - player.y) < 20:
                player.hp -= 20
                enemies.remove(e)

        if player.score > level * 10:
            level += 1

        player.draw()
        for e in enemies:
            e.draw()

        # HUD
        pygame.draw.rect(screen, (40,40,40), (20,20,200,18))
        pygame.draw.rect(screen, GREEN, (20,20,200*(player.hp/100),18))
        screen.blit(FONT.render(f"Score: {player.score}", True, WHITE), (20,45))
        screen.blit(FONT.render(f"Level: {level}", True, WHITE), (WIDTH-120,20))
        screen.blit(FONT.render("Move: WASD / Arrows   Shoot: SPACE",
                                 True, (180,180,180)),
                    (20, HEIGHT-30))

        if player.hp <= 0:
            state = GAME_OVER

    # ================== GAME OVER ==================
    elif state == GAME_OVER:
        screen.blit(BIG.render("GAME OVER", True, RED), (350, 300))
        screen.blit(FONT.render("Press ENTER to return to menu",
                                True, WHITE), (330, 360))

    pygame.display.flip()

pygame.quit()
sys.exit()


ValueError: list.remove(x): x not in list